<img src='https://eurekadocs.readthedocs.io/en/latest/_images/Eureka_logo.png' alt="eureka_logo" width="400px"/><img src='https://exoclimes.org/img/exoslam-bg.png' alt="ariel_france_logo" width="220px"/>

# ExoSLAM 2025
## Setup Notebook - Eureka Data Reduction of MIRI/LRS Data

**Authors**: Taylor James Bell (ESA/AURA for STScI)<br>
**Last Updated**: June 25, 2025<br>
**jwst Pipeline Version**: 1.18.0 (Build 11.3)<br>

**Purpose**:<br>
This setup notebook prepares the necessary files for the Eureka! data analysis tutorial notebook. Data will be located in one observation folder according to the path set up below. It should not be necessary to edit any cells other than in the [Configuration](#1.-Configuration) section.

**Data**:<br/>
This notebook is set up to use an example dataset is from [Program ID](https://www.stsci.edu/jwst/science-execution/program-information) 1366 (PI: Batalha, Natalie) which is the JWST Transiting Exoplanet Community ERS program. In particular, we will use the MIRI/LRS full-orbit phase curve of the hot Jupiter WASP-43b, which was first published in [Bell et al. (2024)](https://ui.adsabs.harvard.edu/abs/2024NatAs...8..879B/abstract). These observations continuously monitored the WASP-43 system for 26.5 hours and contain two eclipses of WASP-43b, one transit of WASP-43b, and the orbital phase variations caused by changes in the regions of WASP-43b's atmosphere that were pointed toward JWST throughout the planet's orbit.

For our purposes, we are only going to work on the first 6 segments of the first exposure to keep things fairly speedy while still getting a realistic sense of what it is like to reduce MIRI/LRS data.

**JWST pipeline version and CRDS context**:<br/>
This notebook was written for the calibration pipeline version given above and uses the context associated with this version of the JWST Calibration Pipeline. Information about this and other contexts can be found in the JWST Calibration Reference Data System (CRDS) [server]((https://jwst-crds.stsci.edu/)). If you use different pipeline
versions, please refer to the table [here](https://jwst-crds.stsci.edu/display_build_contexts/) to determine what context to use. To learn more about the differences for the pipeline, read the relevant [documentation](https://jwst-docs.stsci.edu/jwst-science-calibration-pipeline/jwst-operations-pipeline-build-information)

---
## Table of Contents
* [0. Setting up python environment](#0.-Setting-up-python-environment)
* [1. Configuration](#1.-Configuration)
* [2. Querying MAST using astroquery.mast](#2.-Querying-MAST-using-astroquery.mast)
* [3. Downloading _uncal files for local processing](#3.-Downloading-_uncal-files-for-local-processing)
* [4. Accessing _uncal files from AWS](#4.-Accessing-_uncal-files-from-AWS)
* [5. Checking that the _uncal files downloaded successfully](#5.-Checking-that-the-_uncal-files-downloaded-successfully)
* [6. Copying cached Stage 1 and Stage 4 files for JupyterHub processing](#6.-Copying-cached-Stage-1-and-Stage-4-files-for-JupyterHub-processing)

___
## 1. Configuration

The first step is to setup the notebook and environment.

We'll first import some useful packages.

In [ ]:
import os
import shutil
import numpy as np
from astroquery.mast import Observations

Next, we need to specify where we want to download our data and what data we want to download. The only line you may need to change is the setting of the `path_to_data_folder_on_your_machine` variable if your folder setup is different that was made by default in the Docker setup

In [ ]:
path_to_data_folder_on_your_machine = '../data'  # <--- Please update this variable if needed to match your data location

proposal_id = 1366  # This is the program ID for the JWST Transiting Exoplanet Community ERS program
observation_id = 11  # This is the observation ID for the MIRI/LRS phase curve of WASP-43b
visit_id = 1  # There was only one visit as a part of this observation
exposure_id = 1  # For our purposes, we're only going to download the first exposure
segment_ids = [1, 2, 3, 4, 5, 6]  # For our purposes, we're only going to download the first 6 segments

# For our purposes, we're only going to grab the uncal files
# (0 = raw, 1 = uncalibrated, 2 = calibrated, 3 = science product,
#  4 = contributed science product).
calib_level = 1
# FITS file type, varies by calib_level.
# 1: UNCAL, GS-ACQ1, GS-ACQ2, GS-FG, GS-ID, GS-TRACK
# 2: CAL, CALINTS, RATE, RATEINTS, X1DINTS, ANNNN_CRFINTS,
# GS-ACQ1, GS-ACQ2, GS-FG, GS-ID, GS-TRACK, RAMP
# 3: X1DINTS, WHTLT
subgroup = 'UNCAL'

---
## 2. Querying MAST using astroquery.mast

In [ ]:
# The following code snippet will make sure that the path specified by path_to_data_folder_on_your_machine exists
path_to_data_folder_on_your_machine = os.path.expanduser(path_to_data_folder_on_your_machine)
if path_to_data_folder_on_your_machine[-1] != os.path.sep:
    path_to_data_folder_on_your_machine += os.path.sep
if not os.path.exists(path_to_data_folder_on_your_machine):
    os.makedirs(path_to_data_folder_on_your_machine, exist_ok=True)

# The following code snippet will make a new directory within path_to_data_folder_on_your_machine called Uncalibrated which will store our downloaded _uncal files
uncaldir = os.path.join(path_to_data_folder_on_your_machine, 'Uncalibrated'+os.path.sep)
if not os.path.exists(uncaldir):
    os.makedirs(uncaldir, exist_ok=True)

In [ ]:
# The following code will convert your inputs to properly formatted strings
if type(proposal_id) is not str:
    proposal_id = str(proposal_id).zfill(5)
if type(observation_id) is not str:
    observation_id = str(observation_id).zfill(3)
if type(visit_id) is not str:
    visit_id = str(visit_id).zfill(3)
if type(exposure_id) is not str:
    exposure_id = str(exposure_id).zfill(5)
if type(calib_level) is int:
    calib_level = [calib_level]

# This code will specify the obsid using wildcards. obs_id comes in two flavors
obs_id = f'jw{proposal_id}-o{observation_id}_t*'
obs_id2 = []
for seg_id in segment_ids:
    obs_id2.append(f'jw{proposal_id}{observation_id}{visit_id}_04103_{exposure_id}-seg{seg_id:03}_mirimage')

# Query MAST for requested visit
sci_table = Observations.query_criteria(proposal_id=proposal_id,
                                        obs_id=obs_id)
table = []
if len(sci_table) > 0:
    # Get product list
    data_products_by_id = Observations.get_product_list(sci_table)
    # Filter for desired files
    table = Observations.filter_products(
        data_products_by_id, productSubGroupDescription=subgroup,
        calib_level=calib_level, obs_id=obs_id2)

---
## 3. Downloading _uncal files for local processing

The code cell below downloads the _uncal files from MAST for local processing of the data

In [ ]:
manifest = Observations.download_products(table, download_dir=uncaldir, flat=True)

---
## 4. Downloading _uncal files for JupyterHub/AWS processing

If you are working on AWS, you can download the _uncal FITS files directly from AWS for faster downloads

In [ ]:
# Observations.enable_cloud_dataset()
# manifest = Observations.download_products(table, download_dir=uncaldir, flat=True)

---
## 5. Checking that the _uncal files downloaded successfully
Let's double-check that we have all 6 of the segments we want downloaded and stored in the 'Uncalibrated' folder.

In [ ]:
np.sort(os.listdir(uncaldir))

---